In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Age-Related_Macular_Degeneration"
cohort = "GSE43176"

# Input paths
in_trait_dir = "../../input/GEO/Age-Related_Macular_Degeneration"
in_cohort_dir = "../../input/GEO/Age-Related_Macular_Degeneration/GSE43176"

# Output paths
out_data_file = "../../output/preprocess/Age-Related_Macular_Degeneration/GSE43176.csv"
out_gene_data_file = "../../output/preprocess/Age-Related_Macular_Degeneration/gene_data/GSE43176.csv"
out_clinical_data_file = "../../output/preprocess/Age-Related_Macular_Degeneration/clinical_data/GSE43176.csv"
json_path = "../../output/preprocess/Age-Related_Macular_Degeneration/cohort_info.json"


### Step 1: Initial Data Loading

### Step 2: Dataset Analysis and Clinical Feature Extraction

### Step 3: Gene Data Extraction

### Step 4: Gene Identifier Review

### Step 5: Gene Annotation

### Step 6: Gene Identifier Mapping